# Model Evaluation

In this notebook, we learn about how to use scikit-learn for model evaluation.

<a href="https://colab.research.google.com/github/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/04-model-evaluation.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [ ]:
# Install dependencies for google colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install -r https://raw.githubusercontent.com/thomasjpfan/ml-workshop-intermediate-v2/main/requirements.txt

In [ ]:
import sklearn
assert sklearn.__version__.startswith("1.2"), "Please install scikit-learn 1.2"

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
plt.rcParams['figure.constrained_layout.use'] = True

## Load mammography dataset

In [ ]:
from sklearn.datasets import fetch_openml

mammography = fetch_openml(data_id=310, as_frame=True, parser="pandas")
X, y = mammography.data, mammography.target

In [ ]:
X.head()

In [ ]:
y.value_counts()

In [ ]:
y = (y == '1').astype('int')

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42
)

## Train models for evaluation

### Linear model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [ ]:
log_reg = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Exercise 1

1. Fit a `sklearn.ensemble.RandomForestClassifier` model on the training set.
    - **Hint**: Use `random_state=0`
3. Compute the random forest's predictions on the test set and print the classification report.
4. Compare the classification report of the random forest to logistic regression. Which one has the better overall performance?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/solutions/04-ex01-solutions.py). 

In [ ]:
# %load solutions/04-ex01-solutions.py

**Back to slides!**

## Different Thresholds

### Default

In [ ]:
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

### Probabilities

In [ ]:
y_proba = log_reg.predict_proba(X_test)

In [ ]:
y_proba[65:70]

In [ ]:
y_pred[65:70]

### Threshold at 0.50

In [ ]:
y_pred_50 = y_proba[:, 1] > 0.5
print(classification_report(y_test, y_pred_50))

### Threshold at 0.25

In [ ]:
y_pred_25 = y_proba[:, 1] > 0.25
print(classification_report(y_test, y_pred_25))

In [ ]:
y_pred_75 = y_proba[:, 1] > 0.75
print(classification_report(y_test, y_pred_75))

## Plotting for different thresholds

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
PrecisionRecallDisplay.from_estimator(
    log_reg,
    X_test,
    y_test,
    name="LogisticRegression"
);

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_estimator(
    log_reg,
    X_test,
    y_test,
    name="LogisticRegression"
);

### Use ax to plot both curves next to each other

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
RocCurveDisplay.from_estimator(log_reg, X_test, y_test, name="LogisticRegression", ax=ax1)
PrecisionRecallDisplay.from_estimator(log_reg, X_test, y_test, name="LogisticRegression", ax=ax2);

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
PrecisionRecallDisplay.from_estimator(log_reg, X_test, y_test, ax=ax, name="Logistic Regression")
PrecisionRecallDisplay.from_estimator(rf, X_test, y_test, ax=ax, name="Random Forest");

## Exercise 2

1. Plot the roc curve of the logistic regression model and the random forest model on the same axes.
2. Train a `sklearn.dummy.DummyClassifier()` on the training dataset and plot the precision recall curve and the roc curve with the test dataset.
    - **Hint**: Plot on seperate axes `fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))`
3. What is the ROC AUC and the average precision for the dummy classifer?

In [ ]:
from sklearn.dummy import DummyClassifier
import numpy as np

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/solutions/04-ex01-solutions.py). 

In [ ]:
# %load solutions/04-ex02-solutions.py

## Different metrics
There are many metrics in scikit-lean that can be found
in the [documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#metrics-and-scoring-quantifying-the-quality-of-predictions)

### Metrics that require classes

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
y_pred_log_reg = log_reg.predict(X_test)

In [ ]:
y_pred_log_reg

In [ ]:
f1_score(y_test, y_pred_log_reg)

### Metrics that require ranking

In [ ]:
y_decision_log_reg = log_reg.decision_function(X_test)

In [ ]:
y_decision_log_reg

Aything above 0 is considered class 1:

In [ ]:
np.all((y_decision_log_reg > 0) ==  y_pred_log_reg)

In [ ]:
y_proba_log_reg = log_reg.predict_proba(X_test)

In [ ]:
y_proba_log_reg

#### Aside: Computing the `predict_proba` from the decision function

In [ ]:
from scipy.special import expit

In [ ]:
expit(y_decision_log_reg)

In [ ]:
y_proba_log_reg[:, 1]

### Ranking metrics

In [ ]:
from sklearn.metrics import average_precision_score

#### Using the decision function to compute the average precision

In [ ]:
average_precision_score(y_test, y_decision_log_reg)

#### Using predict_proba to compute the average precision

In [ ]:
average_precision_score(y_test, y_proba_log_reg[:, 1])

## Exercise 3

1. Compute the `roc_auc_score` for the random forest on the test set.
    **Hint**: Use `predict_proba`.
2. Train a `sklearn.svm.SVC` model on the training dataset with `random_state=0`
3. Compute the average precision on the test set.
    - **Hint**: Use `decision_function`.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.svm import SVC

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/solutions/04-ex03-solutions.py). 

In [ ]:
# %load solutions/04-ex03-solutions.py

## Scoring Interface

## Parameter Search with different metrics

### Using a string
Listed in [documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
search = GridSearchCV(
    RandomForestClassifier(random_state=0), 
    param_grid={
        "max_features": [4, 5]
    },
    scoring="average_precision",
    n_jobs=2
)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
search.best_score_

## Scoring interface

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score

In [ ]:
f2_scorer = make_scorer(fbeta_score, beta=2)

In [ ]:
f2_scorer(log_reg, X_test, y_test)

In [ ]:
f2_scorer(rf, X_test, y_test)

### Custom parameters in parameter searching

In [ ]:
search = GridSearchCV(
    RandomForestClassifier(random_state=0), 
    param_grid={
        "max_features": [4, 5]
    },
    scoring=f2_scorer,
    n_jobs=2
)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
search.best_score_

**Back to slides!**

## Inspection 

### Loading housing dataset

In [ ]:
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing(as_frame=True)
X, y = california.data, california.target

X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
feature_names = X.columns

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

### Permutation Importance

In [ ]:
from sklearn.inspection import permutation_importance

rf_perm_results = permutation_importance(
    rf,
    X_test,
    y_test,
    n_repeats=5,
    random_state=0,
)

In [ ]:
rf_perm_results

In [ ]:
def plot_permutation_importance(perm_results, names, top_k=None, ax=None):
    perm_sorted_idx = perm_results.importances_mean.argsort()
    if top_k:
        perm_sorted_idx = perm_sorted_idx[-top_k:]
        
    if ax is None:
        fig, ax = plt.subplots()
    _ = ax.boxplot(perm_results.importances[perm_sorted_idx].T, vert=False,
                   labels=np.array(names)[perm_sorted_idx])
    return ax

In [ ]:
plot_permutation_importance(rf_perm_results, feature_names);

Programmically getting the top 4 features:

In [ ]:
top_features_idx = np.argsort(rf_perm_results["importances_mean"])[::-1]
top_features_idx

In [ ]:
top_4_features_idx = top_features_idx[:4]
top_4_features = feature_names[top_4_features_idx]

In [ ]:
top_4_features

## Exercise 4

1. Create and train a `HistGradientBoostingRegressor` model.
1. Evaluate the model on the test set using `score`.
1. Plot the permutation importance for the gradient boosting model with `n_repeats=5` and `random_state=0`.
1. How does the permtuation feature imporatance compare to random forest?

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/solutions/04-ex04-solutions.py). 

In [ ]:
# %load solutions/04-ex04-solutions.py

**Back to slides!**

## Partial Dependence

In [ ]:
hist = HistGradientBoostingRegressor(random_state=0)

hist.fit(X_train, y_train)

In [ ]:
features = ["Latitude", "MedInc", "AveBedrms", "Population"]

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
PartialDependenceDisplay.from_estimator(hist, X_test, features=features, n_cols=2);

## Housing dataset

In [ ]:
from sklearn.datasets import fetch_openml

ames_housing = fetch_openml(data_id=43926, as_frame=True, parser="pandas")
X, y = ames_housing.data, ames_housing.target

In [ ]:
X.head()

In [ ]:
X.dtypes

In [ ]:
feature_names = X.columns

In [ ]:
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0
)

In [ ]:
ct = ColumnTransformer([
    (
        "numerical",
        "passthrough",
        make_column_selector(dtype_include="number")
    ),
    (
        "category",
         OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
         make_column_selector(dtype_include="number")
    ),
])

In [ ]:
hist = make_pipeline(
    ct,
    HistGradientBoostingRegressor(random_state=0)
)
hist.fit(X_train, y_train)

In [ ]:
hist.score(X_test, y_test)

## Exercise 5

1. Get the permutation importance of the gradient booster (`hist`) using `permutation_importance` and `scoring="neg_mean_absolute_error"`.
    - Store the permutation importance in `hist_perm_results` variable.
    - **Hint**: Use `n_repeats=5` and `random_state=0`.
2. Plot the permutation importance of `hist` by using `plot_permutation_importance` and `top_k=10`.
3. Extract the top 4 features according to permutation importance into a variable `top_4_features`.
    - **Hint**: Use [np.argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) on `hist_perm_results["importance_mean"]` and slice the array to get the top 4 features.
4. Use `PartialDependenceDisplay.from_estimator` to plot the partial dependence of the `top_4_features`.

In [ ]:
import numpy as np

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-v2/blob/main/notebooks/solutions/04-ex05-solutions.py). 

In [ ]:
# %load solutions/04-ex05-solutions.py